In [1]:
import pandas as pd
import os
import openai
import json
import pdb;
import re

In [8]:
SUFFIX = "\n Judgment -->"#'\n\n###\n\n'
#iterate through all the files in index-train.csv
    #capture key tags with regex
    #tags:  {index: (brief-type, outcome, 'train')}
#Beginning
    #iterate through all the files in testset
#End
    # Pandas dataframe of prompt and completion pair

index_train = pd.read_csv('/home/oqcardoso/testset/index-train.csv', header=None)
index_test = pd.read_csv('/home/oqcardoso/testset/index-test.csv', header=None)

tags = {}
for row in index_train.iterrows():
    index_name = re.match(r'.*/(\d+)/.*',row[1][0])[1]
    file_name = re.match(r'.*/.*/.*/(.*[.]txt)',row[1][0])[1]
    if index_name in tags.keys():
        tags[index_name].append((row[1][1], row[1][2], 'train', file_name))
    else:
        tags[index_name] = [(row[1][1], row[1][2], 'train', file_name)]
    
for row in index_test.iterrows():
    index_name = re.match(r'.*/(\d+)/.*',row[1][0])[1]
    file_name = re.match(r'.*/.*/.*/(.*[.]txt)',row[1][0])[1]
    if index_name in tags.keys():
        tags[index_name].append((row[1][1], row[1][2], 'test', file_name))
    else:
        tags[index_name] = [ (row[1][1], row[1][2], 'test', file_name) ]

temp_pandas = {'prompt': [], 'completion': [], 'brief_type': [], 'data_type': [] , 'file_path': [] , 'file_name' : []}
for filename in os.listdir('/home/oqcardoso/testset/files'):
    if filename in tags.keys():
        if filename == '909':
            print("here")
            #pdb.set_trace()
        for tag in tags[filename]:
            brief_type = tag[0]
            outcome = tag[1]
            data_type = tag[2]
            brief = tag[3]
            for file in os.listdir('/home/oqcardoso/testset/files/'+filename):
                if file == brief_type:
                    # for brief in os.listdir('/home/oqcardoso/testset/files/'+filename+'/'+file+'/'):
                    with open('/home/oqcardoso/testset/files/'+filename+'/'+file+'/'+brief, 'r') as f:
                        prompt = f.read()
                        prompt = prompt + SUFFIX
                        completion = ' '+ outcome 
                        temp_pandas['prompt'].append(prompt)
                        temp_pandas['completion'].append(completion)
                        temp_pandas['brief_type'].append(brief_type)
                        temp_pandas['data_type'].append(data_type)
                        temp_pandas['file_name'].append(brief)
                        temp_pandas['file_path'].append(filename)
                        
                    f.close()
                

#convert dictionary to pandas dataframe
df = pd.DataFrame.from_dict(temp_pandas)
df.to_csv('testset.csv')

        

#DC23


here


In [6]:
df

,prompt,completion,brief_type,data_type,file_path,file_name
0,Case 9:11-cv-80416-KLR Document 737 Entered on...,deny,support,train,1013,gov.uscourts.flsd.377721.737.0.txt
1,Case 9:11-cv-80416-KLR Document 743 Entered on...,deny,opposition,train,1013,gov.uscourts.flsd.377721.743.0.txt
2,IN THE UNITED STATES DISTRICT COURT\nFOR THE S...,deny,support,train,909,gov.uscourts.mssd.95610.6.0.txt
3,"I i .\nt | | , : tl\n\n \n\n \n\n \n\n \n\n \n...",deny,opposition,train,909,gov.uscourts.mssd.95610.7.0.txt
4,Case 1:08-cv-11908-RWZ Document 27\n\nFiled 09...,grant,support,train,507,gov.uscourts.mad.118796.27.0.txt
...,...,...,...,...,...,...
1210,Case 6:10-cv-00111-LED Document 333\n\nFiled 0...,deny,opposition,test,621,gov.uscourts.txed.121829.333.0.txt
1211,"Helio LLC v. Palm, Inc. Doc. 3\n\n \n\n \n\n \...",deny,support,train,574,gov.uscourts.cand.187342.3.0.txt
1212,"Helio LLC v. Palm, lnc.\n\n60373/2021440.1\n\n...",deny,opposition,train,574,gov.uscourts.cand.187342.9.0.txt
1213,Case 1:11-cv-00696-RLW Document 4 Filed 04/08/...,grant,support,test,711,gov.uscourts.dcd.147524.4.0.txt


In [7]:
data = {
    'support': [],
    'support_file': [],
    'opposition': [],
    'opposition_file': [],
    'outcome': [],
    'folder_id': [],
    'case_path': [],
    'data_type': []
}

training_ids = []
testing_ids = []

with open('testset/index-train.csv', 'r') as f:
    for line in f:
        result = re.match(r'.*/(\d+)/.*',line)
        training_ids.append(result.group(1))
    f.close()

with open('testset/index-test.csv', 'r') as f:
    for line in f:
        result = re.match(r'.*/(\d+)/.*',line)
        testing_ids.append(result.group(1))
    f.close()


for filename in os.listdir('/home/oqcardoso/testset/files'):
    data['folder_id'].append(filename)
    if filename in training_ids:
        data['data_type'].append('train')
    elif filename in testing_ids:
        data['data_type'].append('test')
    else:
        data['data_type'].append('none')
    with open('/home/oqcardoso/testset/files/'+filename+'/outcome', 'r') as f:
        outcome = f.read()
        result = re.match(r'\d.+(/mnt.+),(.+)', outcome) 
        data['case_path'].append(result.group(1))
        outcome = result.group(2)
        data['outcome'].append(outcome)
        f.close()

    for file in os.listdir('/home/oqcardoso/testset/files/'+filename):
        
        if file in ['support', 'opposition' ]:
            briefs = os.listdir('/home/oqcardoso/testset/files/'+filename+'/'+file+'/')

            if len(briefs) > 1:
                specific_briefs = []
                brief_name = []
                for brief in briefs:
                    with open('/home/oqcardoso/testset/files/'+filename+'/'+file+'/'+ brief, 'r') as f:                                     
                        content = f.read()
                        specific_briefs.append(content)
                        brief_name.append(brief)
                    f.close()
                if file == 'opposition':
                    data['opposition'].append(specific_briefs)
                    data['opposition_file'].append(brief_name)
                elif file == 'support':
                    data['support'].append(specific_briefs)
                    data['support_file'].append(brief_name)
            else:
                with open('/home/oqcardoso/testset/files/'+filename+'/'+file+'/'+briefs[0], 'r') as f:
                    content = f.read()
                    if file == 'opposition':
                        data['opposition'].append([content])
                        data['opposition_file'].append([briefs[0]])
                    elif file == 'support':
                        data['support'].append([content])
                        data['support_file'].append([briefs[0]])
                f.close()


df = pd.DataFrame.from_dict(data)
df.head(10)
df.to_csv('pairedtestset.csv')

In [136]:
openAISchema = ['prompt', 'completion']
supports_train = df[(df['brief_type'] == 'support') & (df['data_type'] == 'train')][openAISchema]
supports_test = df[(df['brief_type'] == 'support') & (df['data_type'] == 'test')][openAISchema]
oppositions_train = df[(df['brief_type'] == 'opposition') & (df['data_type'] == 'train')][openAISchema]
oppositions_test = df[(df['brief_type'] == 'opposition') & (df['data_type'] == 'test')][openAISchema]


In [137]:
supports_train.to_json('supports_train.jsonl', orient='records', lines=True)
supports_test.to_json('supports_test.jsonl', orient='records', lines=True)
oppositions_train.to_json('oppositions_train.jsonl', orient='records', lines=True)
oppositions_test.to_json('oppositions_test.jsonl', orient='records', lines=True)


In [118]:
!openai api fine_tunes.create -t "sport2_prepared_train.jsonl" -v "sport2_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " baseball" -m ada


'files/117/support/gov.uscourts.cand.196504.35.0.txt'

In [140]:
supports_test.iloc[0][1]

' deny'

In [104]:
duplicates.iloc[14]

prompt         \n\nSecurities and Exchange Commission v. Cub...
completion                                                grant
brief_type                                              support
data_type                                                  test
file_name                                                    73
Name: 354, dtype: object

In [109]:
[2023-09-19 17:37:34] Created fine-tune: ft-HU1O5KSY4SyCwJakNMGwEzrL

,prompt,completion,brief_type,data_type,file_name
350,\n\nSecurities and Exchange Commission v. Cub...,grant,support,test,73
351,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
352,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
353,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
354,\n\nSecurities and Exchange Commission v. Cub...,grant,support,test,73
355,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
356,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
357,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
358,\n\nSecurities and Exchange Commission v. Cub...,grant,support,test,73
359,Securities and Exchange Commission v. Cuban\n\...,grant,support,test,73
